In [16]:
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from time import time
import numpy as np
import pandas as pd
import jieba

In [7]:
#read data
data = pd.read_csv('datasets/earphones_300.csv')

#cut data
cw = lambda x: list(jieba.cut(x))
data['content'] = data['content'].apply(cw)

#from list to string...
max_doc_leg = max(len(x) for x in data['content'])
print(max_doc_leg)
texts = [' '.join(x) for x in data['content']]
texts[-2]

1651


'所以 趁着 还 没 爆 ， 赶紧 出手 。'

In [9]:
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = pad_sequences(sequences, maxlen=1700)
sequences = np.array(sequences)

In [10]:
dict_text = tokenizer.word_index
dict_text['所以']

129

In [11]:
sequences[-2]

array([   0,    0,    0, ..., 1360, 2445,    3], dtype=int32)

In [12]:
poslen, neglen = 0, 0
leg = data['sentiment_value']
for i in range(len(data['sentiment_value'])):
    if leg[i] == 1:
        poslen += 1
    else:
        neglen += 1

positive_label = [[0, 1] for _ in range(poslen)]
negative_label = [[1, 0] for _ in range(neglen)]
y = np.concatenate([positive_label, negative_label], 0)

In [13]:
np.random.seed(int(time()))
shuffle_indices = np.random.permutation(np.arange(len(y)))

x_shuffled = sequences[shuffle_indices]
y_shuffled = y[shuffle_indices]

test_sample_index = -1 * int(0.1 * float(len(y)))
x_train, x_test = x_shuffled[:test_sample_index], x_shuffled[test_sample_index:]
y_train, y_test = y_shuffled[:test_sample_index], y_shuffled[test_sample_index:]

In [19]:
sequence_input = Input(shape=(1700, ))
embedding_layer = Embedding(30000, 128, input_length=1700)
embedding_sequences = embedding_layer(sequence_input)

lstm = LSTM(10, dropout=0.2, recurrent_dropout=0.2)(embedding_sequences)
lstm = Dense(16, activation='relu')(lstm)
lstm = Dropout(0.5)(lstm)

pred = Dense(2, activation='softmax')(lstm)
model = Model(sequence_input, pred)

/usr/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [1]:
model.compile(
    optimizer=Adam(),
    loss=categorical_crossentropy, 
    metrics=['accuracy']
)
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test))

NameError: name 'model' is not defined

In [21]:
def I_think(content):
    cw = list(jieba.cut(content))
    word_id = []
    for word in cw:
        try:
            temp = dict_text[word]
            word_id.append(temp)
        except:
            word_id.append(0)
    word_id = np.array(word_id)
    word_id = word_id[np.newaxis, :]
    sequences = pad_sequences(word_id, maxlen=1700, padding='post')
    result = np.argmax(model.predict(sequences))
    if result == 1:
        print(f"Thank you~({result})")
    else:
        print(f"Fuck you!({result})")

In [35]:
I_think('真心不推荐')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Fuck you!(0)
